In [2]:
import numpy as np
import pandas as pd
import mediapipe as mp
import cv2
import pytube
import os
import glob
import pickle
import imageio.v2 as imageio
import json

# Frame 및 Output 생산

In [3]:
def frames_to_gif(frames, output_name):
    images = []
    for frame in frames:
        images.append(imageio.imread(frame))
    imageio.mimsave(output_path+output_name+".gif", images)

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_style = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# TODO: OS 모듈로 영상 별로 이미지 디렉토리 만들고 저장 할 수 있게 변경
video_name = "다만세2.mp4"
video_path = "./" + video_name

# 비디오 로드
cap = cv2.VideoCapture(video_path)
# 프레임 별로 잘린 이미지
frames_path = "./frames/"
# gif
output_path = "./output/"

annotate_frames = []
keypoint_dict = []

# 바로 윈도우 열리고 출력 됨
i = 0
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Empty Frame")
            break
        # 이미지 반전 및 BGR -> RGB
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        
        if results.pose_landmarks is not None:
            annotated_pose_landmarks = {str(j): [lmk.x, lmk.y, lmk.z] for j, lmk in enumerate(results.pose_landmarks.landmark)}
            keypoint_dict.append(annotated_pose_landmarks)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, landmark_drawing_spec=mp_drawing_style.get_default_pose_landmarks_style())
        cv2.imwrite(frames_path+str(i)+".png", image)
        annotate_frames.append(frames_path+str(i)+".png")
        i += 1
        cv2.imshow("Pose KeyPoint Extract: "+video_name, image)
        if cv2.waitKey(5) & 0xFF == 27: break
        
cap.release()
cv2.destroyAllWindows()

# gif 형식으로 저장
frames_to_gif(annotate_frames, video_name)

# 키포인트 json 형식으로 저장
with open(output_path+video_name+"_keypoints.json", "w") as fp:
    json.dump(keypoint_dict, fp)
    
# 대략 2분 정도 걸리는 듯

Empty Frame
